# One-way Repeated Measures ANOVA with post-hoc tests

In this notebook I will demonstrate how to conduct a one-way Repeated Measures (Within-subjects) ANOVA and then conduct post-hoc tests. There are a number of python software libraries that can be used to run this analysis. In this example I will use the pingouin library's rm_anova method and it's pairwise_tests method. 

In [1]:
# Starting with imports of key libraries for the analysis.

import pandas as pd
import numpy as np
import pingouin as pg

Using the EDA polygraph dataset. This is data from a lie detector study with a repeated measures independent variable called questions. This variable has three levels, whether the participant lied, told the truth, or completed a control task in response to each question asked. The dependent variable was the participant's mean arousal recorded using electro-dermal activity (EDA) response when answering with a lie, the truth, or completing the control task (looking at coloured shapes). 

In [2]:
# import the data for analysis.

eda_df = pd.read_csv('EDA Data Long Thin.csv')

eda_df.head()

,PersonID,Framing,Questions_LTCol,Arousal
0,1,1,1,1.277222
1,2,1,1,6.444483
2,3,1,1,1.082127
3,4,1,1,2.704305
4,5,1,1,2.852132


In [3]:
# Conducting the repeated measures ANOVA using the pingouin rm_anova method. 
# The parameters for this method are the data, the DV, the within subjects factor, the subject/ participant ID 
# (to account for multiple datapoints from the same participants), detailed = True returns the result as a pandas dataframe.

res = pg.rm_anova(data = eda_df, dv = 'Arousal', within = 'Questions_LTCol', subject = 'PersonID', detailed = True)

res

,Source,SS,DF,MS,F,p-unc,p-GG-corr,ng2,eps,sphericity,W-spher,p-spher
0,Questions_LTCol,48.467643,2,24.233822,14.216746,0.000002,0.000004,0.045598,0.923963,False,0.917706,0.035111
1,Error,269.326317,158,1.704597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see from the above output that the assumption of sphericity has been violated (W = 0.92, p = 0.035), so in this case we need a robust test of significance. In the output pingouin has provided us with a p-value that has been corrected for violations of sphericity using the Greenhouse Geisser method (p-GG-corr). Using this adjusted p-value we see that the ANOVA result is still significant. Greenhouse Geisser corrected ANOVA: F(2, 158) = 14.22, p < 0.001. 

In [4]:
# As we have a significant ANOVA result we would now want to perform post-hoc comparisons on all pairs of conditions 
# of the independent variable (questions). We can do this using pingouin's pairwise_tests method. Below I have used Bonferroni
# corrected pairwise comparisons.

post_res = pg.pairwise_tests(data = eda_df, dv = 'Arousal', within = 'Questions_LTCol', subject = 'PersonID', 
                            parametric = True, padjust = 'bonf', effsize = 'hedges')

post_res

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Questions_LTCol,1,2,True,True,1.126317,79.0,two-sided,0.263441,0.790323,bonf,0.227,0.094642
1,Questions_LTCol,1,3,True,True,4.482561,79.0,two-sided,0.000025,0.000074,bonf,728.105,0.522986
2,Questions_LTCol,2,3,True,True,4.095714,79.0,two-sided,0.000101,0.000303,bonf,196.816,0.397882


Descriptive statistics for each of the levels of the IV are also helpful to interpret the direction of any significant comparisons.

To obtain these I am first going to label the Questions variable and then obtain descriptive statistics (mean arousal scores) for the three levels separately.

In [5]:
# Using the series.map function to add labels to the Questions_LTCol variable.

a = [1, 2, 3]
b = ["Lie", "Truth", "Col"]

eda_df['ques_cat'] = eda_df['Questions_LTCol'].map(dict(zip(a, b)))

eda_df

,PersonID,Framing,Questions_LTCol,Arousal,ques_cat
0,1,1,1,1.277222,Lie
1,2,1,1,6.444483,Lie
2,3,1,1,1.082127,Lie
3,4,1,1,2.704305,Lie
4,5,1,1,2.852132,Lie
...,...,...,...,...,...
235,76,2,3,2.411927,Col
236,77,2,3,1.339430,Col
237,78,2,3,-0.197783,Col
238,79,2,3,-0.040323,Col


In [6]:
# Creating new objects for the IV and DV. 
cat = eda_df['ques_cat']
scale = eda_df['Arousal']

In [7]:
# Next creating three boolean objects spltting cat up by level/ condition.

cat_1 = cat == 'Lie'
cat_2 = cat == 'Truth'
cat_3 = cat == 'Col'

In [8]:
# Finally create a list of each arousal score broken down by level. Using the above booleans.

cat_lie_arousal = scale[cat_1]
cat_truth_arousal = scale[cat_2]
cat_col_arousal = scale[cat_3]

In [9]:
# Obtaining descriptive statistics for the above three levels. Using the print function. 

print(f"Mean Lie Arousal: {cat_lie_arousal.mean():.2f}")
print(f"Mean Truth Arousal: {cat_truth_arousal.mean():.2f}")
print(f"Mean Control Arousal: {cat_col_arousal.mean():.2f}")

Mean Lie Arousal: 2.09
Mean Truth Arousal: 1.88
Mean Control Arousal: 1.05


The levels of the questions variable have not been labelled in this dataset but they are 1 - Lie, 2 - Truth, 3 - Coloured Squares (Control). 
These numbers for the variable levels relate to the contrast between the columns labelled A and B.

From this set of comparisons we can see that: 

1 - Lie (Mean = 2.09) and 2 - Truth (Mean = 1.88) are not significantly different (t(79) = 1.13, p = 0.79). Using Hedges' g this represented a small effect size (g = 0.09) 

1 - Lie (Mean = 2.09) and 3 - Coloured Squares (Mean = 1.05) are significantly different (t(79) = 4.48, p < 0.001). This represented a medium effect size (Hedges' g = 0.52). 

2 - Truth (Mean = 1.88) and 3 - Coloured Squares (Mean = 1.05) are significantly different (t(79) = 4.10, p < 0.001). This represented a small to medium effect size (Hedges' g = 0.40). 

So, the Lie and Truth conditions showed significantly higher mean EDA arousal than the Coloured Squares (control) condition. 